<h4>Lily Grumbach
    <br>M1 Humanités numériques - Université PSL</h4>
<h1><center>Rendu TAIS-TAL </center></h1>
<h2><center>Partie TAIS</center></h2>
<h3><center>2/5 : Extraire des bases de données à ma disposition les coordonnées des lieux et créer un dictionnaire de désambiguisation de mes données</center></h3>

<h3><u>PLAN:</u></h3>

1) Harmonisation des données Géopolitiques pour le fond de carte de 1914
<br/>
    
<h4> 2) Explorer les bases de données à ma disposition (IREL et GPH) et les préparer pour l'extraction des coordonnées géographiques </h4>
 
3) Reconstituer les liens URL de la base de données IREL des Archives nationales d'Outre-mer.

4) Reconstituer les coordonnées géographiques des GPH et de l'IREL
 
5) Géoréférencer les données de mes revues
<b>Stratégie en place :  </b>

Nous avons 3 sources possibles de désambiguisation : 
* **Wikidata** indiqués par le medialab 
* l'**IREL**, la base de données géographique de l'inventaire en ligne des Archives nationales d'Outre Mer
* **Geonames**, pour le meilleur et pour le pire.

<h4>Objectifs de la partie : </h4>
Faire une estimation de ce que je pourrai récupérer avec les données bases de données que j'ai et les préparer pour l'extraction de coordonnées géographiques

# Import des modules

In [29]:
#module
import desambiguisation

##General : 
from desambiguisation import ListEntities2df
##GPH
from desambiguisation import MatchGPELOC_IREL,Extract_longlat_WikiData_ALL
##IREL : 
from desambiguisation import nettoyage_df_IREL,urlencode,Extract_longlat_IREL,IREL_Nettoyage_AdminLieuDit,nettoyage_desambiguisation,EgaliserTaille_MatchNot_IREL

#Généraux
import pandas as pd
import numpy as np
import re

#Webscrapping
from bs4 import BeautifulSoup
import geocoder
from lxml import html
import requests

#Wikidata 
import qwikidata

# 0. Les df de référence

**AHMC**

In [2]:
##AHMC
df_AHMC_annote = pd.read_csv('./df_annotes/dfAHMCannotations.csv')

#Ne conserver qu'une partie du df
df_AHMC_annote = df_AHMC_annote[["article_titre","revue_annee","GPE_ents","LOC_ents","ORG_ents"]]

#Extraire les entités GPE,LOC,ORG
liste_GPEAHMC = ListEntities2df(df_AHMC_annote,"AHMC","GPE_ents")
liste_LOCAHMC = ListEntities2df(df_AHMC_annote,"AHMC","LOC_ents")
liste_ORGAHMC = ListEntities2df(df_AHMC_annote,"AHMC","ORG_ents")

fichier créé dans ./desambiguisation/Revue_ParEntityLabel/AHMC/AHMCGPE_ents.csv
Output:liste des entités GPE_ents 

fichier créé dans ./desambiguisation/Revue_ParEntityLabel/AHMC/AHMCLOC_ents.csv
Output:liste des entités LOC_ents 

fichier créé dans ./desambiguisation/Revue_ParEntityLabel/AHMC/AHMCORG_ents.csv
Output:liste des entités ORG_ents 



**AMN**

In [3]:
#AMN
df_AMN_annote = pd.read_csv('./df_annotes/dfAMNannotations.csv')

#Ne conserver qu'une partie du df
df_AMN_annote = df_AMN_annote[["article_titre","revue_annee","GPE_ents","LOC_ents","ORG_ents"]]

#Extraire les entités GPE,LOC,ORG
liste_GPEAMN = ListEntities2df(df_AMN_annote,"AMN","GPE_ents")
liste_LOCAMN = ListEntities2df(df_AMN_annote,"AMN","LOC_ents")
liste_ORGAMN = ListEntities2df(df_AMN_annote,"AMN","ORG_ents")

fichier créé dans ./desambiguisation/Revue_ParEntityLabel/AMN/AMNGPE_ents.csv
Output:liste des entités GPE_ents 

fichier créé dans ./desambiguisation/Revue_ParEntityLabel/AMN/AMNLOC_ents.csv
Output:liste des entités LOC_ents 

fichier créé dans ./desambiguisation/Revue_ParEntityLabel/AMN/AMNORG_ents.csv
Output:liste des entités ORG_ents 



# 1. Désambiguisation de la liste des GPE et LOC

In [4]:
list_GPELOC = liste_GPEAMN+liste_LOCAMN+liste_LOCAHMC+liste_GPEAHMC
print("Nombre de lieux à annoter : ",len(list_GPELOC))

Nombre de lieux à annoter :  260


Dictionnaire et liste des Faux positifs relevés dans la liste :

In [5]:
#dictionnaire pour la désambiguisation :
Dico_desambiguisation = {"Guinée Française":["Guinée française"],
                         "Côte d'Ivoire":["côte d'Ivoire"],
                         "Pak-Hoi":["Pak-Hoï","Pakhoï"],
                         "Chengdu":['Tchen-Tou','Tchentou'],
                         "Yunnan":['Yun-Nam','Yun-Nan','Yunnam'],
                         "Hanoi":["Hanoï"],
                         "Laokay":["Lao-kay"],
                         "Côte d'Ivoire":["Côte d\'Ivoire","côte d'Ivoire"]
                        }
Liste_FP = ["Annamite","Annamites","île"]

In [6]:
#désambiguiser :
for i in range(len(list_GPELOC)):
    for (key,val) in Dico_desambiguisation.items():
            for name in val:
                if list_GPELOC[i]==name :
                    list_GPELOC[i]=key

#retirer lesfaux positifs :
for FP in Liste_FP:
    print(FP)
    list_GPELOC.remove(FP)
# list_GPELOC

Annamite
Annamites
île


In [7]:
#Retirer les doublons de la liste :
print("avant dédoublonnage : ",len(list_GPELOC))
list_GPELOC = list(dict.fromkeys(list_GPELOC))
print("après dédoublonnage : ",len(list_GPELOC))

avant dédoublonnage :  257
après dédoublonnage :  225


In [8]:
#Mise en df pour exploitation plus tard
df_GPELOC = pd.DataFrame({"NAME":list_GPELOC})

Taille finale des entités à annoter : 

In [9]:
df_GPELOC

,NAME
0,Acores
1,Algérie
2,Allemagne
3,Angleterre
4,Annam
...,...
220,lac Tchad.
221,île de Saint-Barthélémy
222,île de Saint-Martin
223,île de la Réunion


Exportation pour exploitation future : 

In [10]:
df_GPELOC.to_csv("./output/2-Exploration_GPH-IREL/GPELOC_à_annoter.csv",index=False)

# 2. Explorer les possibilités entre GPH, IREL et GPELOC

## 2.1. Importation et première exploration des BDD
NB : je n'applique cela qu'aux BDD GPH et IREL car Geonames est beaucoup trop volumineux.

### 2.1.1. GPH 
Import de la base de données du medialab contenant les liens wikidata

In [11]:
#import du df GPH entities avec lien wikidata
df_wikiData = pd.read_csv("./GeoPolHist-202103/medialab-GeoPolHist-fb19b66/data/GeoPolHist_entities.csv")
df_wikiData=df_wikiData.rename(columns={"GPH_name":"NAME"})

Import des données de 1914 nettoyées et enrichies manuellement pour correspondre au fond de carte de QGIS. On merge ce df avec celui contenant les liens wikidata :

In [12]:
#Importer le df de référence (jusqu'à 1914)
df_MANUEL = pd.read_csv("./output_finaux/df_1914manuelCarto.csv")

#Merge avec les données complétées manuellement 
df_WikiQGIS=pd.merge(df_wikiData,df_MANUEL,how="right",on="GPH_code")

#Export : 
df_WikiQGIS.to_csv("./output/2-Exploration_GPH-IREL/WikiQGIS.csv")

###  2.1.2. IREL

Nettoyer la base de donnée IREL (cf module)

In [13]:
dico_IREL ={}
n=0

with open('./IREL/liste_lieux_IREL.js','r') as f:
    lignes = f.readlines()
    
    for l in lignes:
        l=l.split("\n")
        dico_IREL[n]=l
        n+=1

df_IREL = nettoyage_df_IREL(dico_IREL)

return: df_IREL nettoyé


### 2.1.3. Merge df_IREL et df_WikiData pour avoir les lieux présents dans les deux bases de données

In [14]:
#Renommer les colonnes pour pouvoir merge :
df_IREL=df_IREL.rename(columns={"Lieu-dit":"NAME"})
df_wikiData=df_wikiData.rename(columns={"GPH_name":"NAME"})

df_WikiIREL = pd.merge(df_wikiData,df_IREL,how="inner",on="NAME")

#ATTENTION : je drop les duplicatas sur les GPH_code pour avoir une idée plus géographique qu'historique:
df_WikiIREL["GPH_code"]=df_WikiIREL["GPH_code"].drop_duplicates(keep="first")

#Ici onsouhaite avoirleurnombredans l'absolu,regardless of the GPH Status
df_WikiIREL=df_WikiIREL.loc[df_WikiIREL["GPH_code"].notnull()]

### 2.1.4.  Explorations de nos df :

=> visualisation du nombre d'entité par df

In [15]:
df_comparaison_DF = pd.DataFrame({"df_IREL":len(df_IREL),"df_wikiData":len(df_wikiData),"df_WikiQGIS":len(df_WikiQGIS),"df_WikiIREL":len(df_WikiIREL),"df_GPELOC":len(df_GPELOC)},index=["Nombre d'entités"])
df_comparaison_DF

,df_IREL,df_wikiData,df_WikiQGIS,df_WikiIREL,df_GPELOC
Nombre d'entités,14571,1228,203,87,225


Avoir une idée du nombre d'entités que je peux **à priori** (pour le meilleur comme pour le pire) trouver dans les deux bases de données : 

In [ ]:
df_GPELOC=df_GPELOC.rename(columns={"entités à annoter":"NAME"})

#Nombre d'entités de GPELOC trouvable dans GPH: 
GPELOC_GPH =pd.merge(df_wikiData,df_GPELOC,how="inner",on="NAME")[["NAME"]]

#Nombre d'entités de GPELOC trouvable dans IREL: 
GPELOC_IREL =pd.merge(df_IREL,df_GPELOC,how="inner",on="NAME")[["NAME"]]

#Nombre d'entités de GPELOC dans IREL *et* das GPH : 
GPELOC_IRELGPH =pd.merge(df_WikiIREL,df_GPELOC,how="inner",on="NAME")[["NAME"]]

#Nombre d'entités de GPELOC dans GPH en 1914 :
GPELOC_GPH1914 =pd.merge(df_WikiQGIS,df_GPELOC,how="inner",on="NAME")[["NAME"]]

df_comparaison_GPELOC = pd.DataFrame({"avec GPH":len(GPELOC_GPH),"avec IREL":len(GPELOC_IREL),"IRELxGPH":len(GPELOC_IRELGPH),"GPH en1914":len(GPELOC_GPH1914)},index=["Nombre d'entités"])
df_comparaison_GPELOC

In [ ]:
liste_ent_trouvables = GPELOC_GPH['NAME'].tolist()+GPELOC_IREL['NAME'].tolist()
len(liste_ent_trouvables)
# GPELOC_IREL['NAME'].tolist()

**A l'issue de cette première étude, on en conclut la stratégie d'action suivante:**

   1) Trouver les `23 GPELOC` de la base de données `GPH` => Les retirer de la liste de recherche des GPELOC

   2) Dans `IREL`, chercher les 103-11=`92 noms` de lieux restant 

   3) Chercher les 225-126 = `99` entités restantes avec `Geonames`

## Exportation du df_IREL pour exploitation dans le prochain notebook : 

In [ ]:
df_IREL.to_csv("./output/2-Exploration_GPH-IREL/IREL_nettoye.csv")